<img src="figures/icon-512x512-brazildatacube-png.png" alt="Brazil Data Cube logo" align="right" width="172" />

## <span style="color:#336699">Evaluating  Landsat-8  and  Sentinel-2  Nadir  BRDF  Adjusted  Reflectance (NBAR) on South of Brazil through a Reproducible and Replicable Workflow</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Rennan de Freitas Bezerra Marujo<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe M. Carlos<sup><a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Raphael W. Costa<sup><a href="#"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Carlos A. F. Noronha<sup><a href="#"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, José Guilherme Fronza<sup><a href="https://orcid.org/0000-0002-0830-8101"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Jeferson de Souza Arcanjo<sup><a href="https://orcid.org/0000-0002-0830-8101"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Anderson Soares<sup><a href="https://orcid.org/0000-0001-6513-2192"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Karine R. Ferreira<sup><a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
    <b>Abstract.</b>This notebook presents the step-by-step execution of the workflow of the <i>Evaluating  Landsat-8  and  Sentinel-2  Nadir  BRDF  Adjusted  Reflectance (NBAR) on South of Brazil through a Reproducible and Replicable Workflow</i> article.
</div>   

<br>

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement to the <a href="#">Section 3</a> of the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
      <p> Marujo, R. F.B.; Carlos, F.M.; Costa, R.W.; Noronha, C.A.F.; Arcanjo, J.S.; Fronza, J.G.; Soares, A.R.; Queiroz, G.R.; Ferreira, K.R; 2021. Evaluating  Landsat-8  and  Sentinel-2  Nadir  BRDF  Adjusted  Reflectance (NBAR) on South of Brazil through a Reproducible and Replicable Workflow</p>
    </div>
</div>

### Comparing Landsat-8/OLI and Sentinel-2/MSI Surface images (SR) and Nadir BRDF Adjusted Reflectance (NBAR).
<hr style="border:1px solid #0077b9;">

This paper presents the processing workflow used to compare surface reflectance (SR) images from the Landsat-8/OLI and Sentinel-2/MSI. In this workflow, the first step is performing atmospheric correction of the Sentinel-2/MSI images. Afterward, in the second step, the NBAR correction on the Landsat-8/OLI and Sentinel-2/MSI images is done. Finally, after the data are generated, they are used to perform the comparisons presented in the paper.

> The codes presented in this notebook are the same ones used for generating the results presented by Marujo *et al.* (2021).


### Study Area
<hr style="border:1px solid #0077b9;">

This study used Sentinel-2/MSI, and Landsat-8/OLI images for the Rio Grande do Sul state region in the south of Brazil. In this region, there is the presence of overlapping Landsat-8/OLI and Sentinel-2/MSI orbits. This overlap is beneficial for the article comparisons since the border of the images, where the overlap occurs, is more affected by view angle variation. In **Figure 1**, the study region is shown, at WRS2 path/rows `223081` and `223082` and at MGRS Granule `22JBM`.

<div align="center">
  <img src="figures/study_area.png" alt="Study Area" width="600px"/>
   
  <div align="center">
      <b>Figure 1.</b> Study Area
  </div>
</div>

### 1. Resource configuration
<hr style="border:1px solid #0077b9;">

The first step of the Workflow consists in configuring the input directories, images to be processed, auxiliary data and output directories.

In [1]:
import os

from research_processing import toolbox

#### 1.1 Configure data directory (with environment variable)

Initially, we will configure the workflow base directory. In this Jupyter Notebook, we will set this directory based on the `DATA_DIRECTORY` environment variable, representing the root directory of the **R**esearch **C**ompendium (RC) located on the user's machine.

> If you are running this notebook in the Docker environment provided in RC, the directory pointed on the `DATA_DIRECTORY` will be mapped as a Docker volume. Please refer to the [RC documentation](#) provided by Marujo *et al.* (2021) for more information on setting up the environment.

In [2]:
data_directory = os.environ.get("DATA_DIRECTORY")
data_directory

'/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data'

#### 1.2 Defining input files and directories for Sentinel-2/MSI data

Now, let's define the path to the directory where the `.SAFE` directories with the Sentinel-2 L1C (Level 1 C, Top of Atmosphere) data that will be used are located.

> The data used in this study was obtained from [Copernicus Scihub](https://scihub.copernicus.eu/dhus/#/home).


In [3]:
sentinel2_input_dir = os.path.join(data_directory, "raw_data/Sentinel2Data")
sentinel2_input_dir

'/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/raw_data/Sentinel2Data'

In this notebook, the definition of which images are to be used is done through a `txt` file. All the scene ids (.SAFE directory names) to be processed are placed in this file.

> The name specified in the `txt` file must be the same as the directory where the Sentinel-2/MSI scenes are located since it will be used as the basis for selecting the scene and loading the data.

The code below defines the `txt` file that specifies the Sentinel-2/MSI data used:

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 95%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;"> 
    <b>Important note</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
        <p>If you do not have the file used below, please refer to the <a href="#">RC documentation</a>. In the documentation, instructions are available to help you set up the environment needed to run this notebook.</p>
</div>


In [4]:
sentinel2_sceneid_list = os.path.join(data_directory, "raw_data/ExampleFilesList/s2-sceneids.txt")
sentinel2_sceneid_list

'/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/raw_data/ExampleFilesList/s2-sceneids.txt'

With the file defined, we can read it. When doing this reading, some special characters may be present in the file's content. We must remove these characters to avoid problems during the processing steps. So in the code below, the file is loaded, and then its content is used as input to the function `research_processing.toolbox.standardize_filename`, which removes possible special line break characters:

In [5]:
sentinel2_sceneids = toolbox.standardize_filename(open(sentinel2_sceneid_list).readlines())
sentinel2_sceneids[0:3]  # Showing the first three

['S2B_MSIL1C_20171119T133209_N0206_R081_T22JBM_20171120T175608.SAFE',
 'S2B_MSIL1C_20171122T134159_N0206_R124_T22JBM_20171122T200800.SAFE']

#### 1.3 Defining input files and directories for Landsat-8/OLI steps

As in [Section 2.4](##1.2-Defining-input-files-and-directories-for-Sentinel-2/MSI-data), for the Landsat-8/OLI data, we first define the directory where the directories of the Landsat-8 Collection-2 Level-2 (Surface Reflectance) scenes are stored.

> The Landsat-8 Collect-2 Level 2 data used in this work was originally retrieved through the [USGS Earth Explorer](http://earthexplorer.usgs.gov/).


In [6]:
landsat8_input_dir = os.path.join(data_directory, "raw_data/Landsat8Data")
landsat8_input_dir[0:3]  # Showing the first three

'/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/raw_data/Landsat8Data'

To specify what Landsat-8/OLI scenes will be processed, the code below defines the `txt` file that contains the name of each used scene (Directory Name):

> The name specified in the `txt` file must be the same as the directory where the Landsat-8/OLI scenes are located since it will be used as the basis for selecting the scene and loading the data.

In [7]:
landsat8_sceneid_list = os.path.join(data_directory, "raw_data/ExampleFilesList/l8-sceneids.txt")
landsat8_sceneid_list

'/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/raw_data/ExampleFilesList/l8-sceneids.txt'

Now, analogous to the process in [Section 2.4](##1.2-Defining-input-files-and-directories-for-Sentinel-2/MSI-data), the file content will be loaded and then used as input for the `toolbox.standardize_filename` function, removing possible line break characters from the file contents. These special characters cause problems during code execution.

In [8]:
landsat8_sceneids = toolbox.standardize_filename(open(landsat8_sceneid_list).readlines())
landsat8_sceneids[0:3]  # Showing the first three

['LC08_L2SP_222081_20171120_20200902_02_T1',
 'LC08_L2SP_223081_20171111_20200902_02_T1']

#### 1.4 Defining LaSRC auxiliary data

The Sentinel-2 L1C will be processed to Surface Reflectance through LaSRC and Sen2cor Atmospheric Correction. However, LaSRC requires a few auxiliary files. We can obtain these auxiliary files from the directory [lasrc_auxiliary/L8/](https://edclpdsftp.cr.usgs.gov/downloads/auxiliaries/lasrc_auxiliary/L8/), provided by the USGS. From this directory, we need all the data to perform the atmospheric correction. The code below defines the path to the directory where this data is stored in the RC:

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 95%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;"> 
    <b>Important note</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
        <p>Among the files made available in the auxiliary directory are the LADS data which are daily files required to perform the atmosphere correction. This daily frequency makes this directory a considerable size. Therefore, you can acquire only the files that refer to the dates of the scenes you will process.</p>
        <p>Only the LADS data from the scenes used in the minimal example are being used in this RC.</p>
    </div>
</div>
<br>

In [9]:
lads_auxiliary_data = os.path.join(data_directory, "raw_data/LaSRC_AuxiliaryData/")
lads_auxiliary_data

'/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/raw_data/LaSRC_AuxiliaryData/'

#### 1.5 Defining the base output directory

The results of the processing steps that we will perform next on this notebook, which includes the processed images and the results of the comparisons, will be saved in separate directories, following the structure shown below:

```
BASE_OUTPUT_DIRECTORY
├── l8
├── s2
└── analysis
```

Where:

1. **l8**: Directory with the processed Landsat-8/OLI data;
2. **s2**: Directory with the processed Sentinel-2/MSI datas;
3. **analysis**: Directory with the comparisons results.

Therefore, it is necessary to define a base directory, which will store all other directories. In the code below, this directory is defined:

In [10]:
derived_data_dir = os.path.join(data_directory, "derived_data")
derived_data_dir

'/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/derived_data'

Finally, let's define the other directories:

In [11]:
#
# Landsat-8/OLI output directory
#
l8_outputs = os.path.join(derived_data_dir, "l8")

#
# Sentinel-2/MSI output directory
#
s2_outputs = os.path.join(derived_data_dir, "s2")

#
# Comparison analysis results
#
analysis_outputs = os.path.join(derived_data_dir, "analysis")

The next sections will make use of these directories as a basis for storing the generated results.

### 2. Atmosphere Correction
<hr style="border:1px solid #0077b9;">

For this study, Landsat-8 images were already acquired as Surface Reflectance products. However, Sentinel-2 images were obtained as L1C (Top of Atmosphere). To compare Sentinel and Landsat-8 first we must process Sentinel-2 to Surface Reflectance. For that we opted for using 2 methods:

- `Sen2cor`: is provided by the European Space Agency (ESA) and consist in a processor for Sentinel-2 Level 2A product generation

- `LaSRC`: The Landsat 8 Collection 1 Land Surface Reflectance Code (LaSRC) software uses an implementation of the Second Simulation of a Satellite Signal in the Solar Spectrum (6S) atmospheric correction model, to correct the atmospheric effects. It was originally projected to correct Landsat-8 Collection-1 images, but was adapted to also runs Sentinel-2 images.

On this section we perform the atmosphere correction with sen2cor.

#### 2.1. Sentinel-2/MSI (Sen2Cor)

To process the Sentinel-2 L1C images using `sen2cor`, we will use the features implemented in the `research processing` library provided in the article RC. Then, let's import from this library the function `sen2cor`, which performs the atmospheric correction of Sentinel-2/MSI scenes:

In [12]:
from research_processing.surface_reflectance import sen2cor


<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 95%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;"> 
    <b>Important note</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
        <p>If you do not have the <emph>research processing</emph> library in your environment, please, refer to the <a href="#">RC documentation</a> for more information. In the documentation, the technical details of each part of this RC are available, which includes the step-by-step setup of the complete environment for running the codes.</p>
    </div>
</div>
<br>

#### 2.1.1 Preparing the sen2cor output directory

Using the base output directory previously created to store the results generated with Sentinel-2/MSI data, in the code below is defined the directory where the atmospheric correction results will be saved:

In [13]:
sen2cor_dir = toolbox.prepare_output_directory(s2_outputs, "s2_sen2cor_sr")
sen2cor_dir

#### 2.1.2 Applying sen2cor

Now we can perform the atmosphere correction using `Sen2cor`:

> Atmospheric correction using sen2cor can take considerable time. For the production of this notebook, we used a computer with an Intel Xeon processor and 48 GB RAM. In this scenario, the processing took 20 minutes per image.

In [14]:
s2_sen2cor_outputs = sen2cor(sentinel2_input_dir, sen2cor_dir, sentinel2_sceneids)
s2_sen2cor_outputs[0:3]  # Showing the first three

['/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/derived_data/s2/s2_sen2cor_sr/S2B_MSIL1C_20171119T133209_N0206_R081_T22JBM_20171120T175608.SAFE',
 '/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/derived_data/s2/s2_sen2cor_sr/S2B_MSIL1C_20171122T134159_N0206_R124_T22JBM_20171122T200800.SAFE']

#### 2.1.3 Saving Sen2Cor scene list

Finally, save the name of each of the processed scenes. This step is necessary for two reasons: (I) We will use the names in subsequent steps to determine which Sentinel-2/MSI images with atmospheric correction should be used; (II) The names of the atmospherically corrected scenes are different from the original ones because of the processing date.


In [15]:
sen2cor_sceneids = toolbox.filename(os.listdir(sen2cor_dir))
sen2cor_sceneids[0:3]  # Showing the first three

### 2.2. Sentinel-2/MSI (LaSRC)

As performed using `Sen2cor`, here we atmospherically correct the Sentinel-2 L1C to produce Surface Reflectance products, but correcting through LaSRC. Then, just as with the `sen2cor` function, we will import the `lasrc` function available in the `research processing` library:

In [16]:
from research_processing.surface_reflectance import lasrc

#### 2.2.1 Preparing the LaSRC output directory

As in [Section 2.1.1](#2.1.1-Preparing-the-sen2cor-output-directory), let's create the directory where we will save the Sentinel-2/MSI data corrected with LaSRC:

In [17]:
lasrc_dir = toolbox.prepare_output_directory(s2_outputs, "s2_lasrc_sr")
lasrc_dir

#### 2.2.2 Applying LaSRC

Now we can perform the atmosphere correction using `LaSRC`:

> Atmospheric correction using sen2cor can take considerable time. For the production of this notebook, we used a computer with an Intel Xeon processor and 48 GB RAM. In this scenario, the processing took 20 minutes per image.

In [18]:
s2_lasrc_outputs = lasrc(sentinel2_input_dir, lasrc_dir, sentinel2_sceneids, lads_auxiliary_data)
s2_lasrc_outputs[0:3]  # Showing the first three

['/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/derived_data/s2/s2_lasrc_sr/S2B_MSIL1C_20171119T133209_N0206_R081_T22JBM_20171120T175608.SAFE',
 '/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/derived_data/s2/s2_lasrc_sr/S2B_MSIL1C_20171122T134159_N0206_R124_T22JBM_20171122T200800.SAFE']

#### 2.2.3 Saving LaSRC scene list

Using the same approach as in [Section 2.1.3](#2.1.3-Saving-Sen2Cor-scene-list), we will save the name of the scenes that were generated with LaSRC:

In [19]:
lasrc_sceneids = toolbox.filename(os.listdir(lasrc_dir))
lasrc_sceneids[0:3]  # Showing the first three

### 3. NBAR correction
<hr style="border:1px solid #0077b9;">

Remote sensing systems observations are influenced by the Sun-Sensor Geometry Effects, which are described by the Bidirectional Reflectance Distribution Function (BRDF). The surface reflectance measured by an orbital sensor is influenced by changes on the Sun-Sensor geometry due to differences in azimuth and zenith angles. Scattering models can be used to estimate Bidirectional Reflectance Factors (BRFs) as if they were observed in a specific view and sun zenith angles. Nadir BRDF-Adjusted Reflectance (NBAR) products are surface reflectance estimated through a scattering model at nadir (0&deg; view zenith) and a specified solar zenith angle.

On this section we perform the NBAR correction on Sentinel-2/MSI (with sen2cor atmosphere correction) and Landsat-8/OLI images.

#### 3.1. Sentinel-2/MSI (Sen2Cor) NBAR

To perform the NBAR correction on Sentinel-2/MSI and Landsat-8/OLI images, we will use the features implemented in the `research processing` library provided in the RC of the article. Since NBAR correction can be done for both Sentinel and Landsat images, there are three functions for performing the correction in the `research processing` library, one for each satellite-sensor and atmosphere corrections (LaSRC e Sen2cor). This section will process Sentinel data, so we start by importing the function `s2_sen2cor_nbar`, which applies the NBAR correction to Sentinel-2/MSI scenes. The code below imports this function from the library:

In [20]:
from research_processing.nbar import s2_sen2cor_nbar

#### 3.1.1 Preparing the Sentinel-2/MSI (Sen2cor) NBAR correction output directory

The code below defines the output directory for the data generated in the NBAR correction. This output directory will be based on the Sentinel output directory, specified and used in the previous steps to store the generated Sentinel products.

In [21]:
s2_sen2cor_nbar_dir = toolbox.prepare_output_directory(s2_outputs, "s2_sen2cor_nbar")
s2_sen2cor_nbar_dir

#### 3.1.2 Generate NBAR for Sentinel-2/MSI (Sen2cor) products

Now we can generate the NBAR on the Surface Reflectance products (generated through Sen2cor).

> To generate Sentinel-2 NBAR, the function `s2_sen2cor_bar` uses the [s2-angs](https://github.com/brazil-data-cube/s2-angs) python library. Due to that, here, we do not need to call the angle generation explicitly.

In [22]:
s2_sen2cor_nbar_outputs = s2_sen2cor_nbar(sen2cor_dir, s2_sen2cor_nbar_dir, sen2cor_sceneids)
s2_sen2cor_nbar_outputs[0:3]  # Showing the first three

['/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/derived_data/s2/s2_sen2cor_nbar/S2B_MSIL2A_20171119T133209_N9999_R081_T22JBM_20211023T084344.SAFE',
 '/home/felipe/Mestrado/04_ExperimentoHarmonizacao/Evaluating-Landsat-8-and-Sentinel-2-Nadir-BRDF-Adjusted-Reflectance/analysis/data/derived_data/s2/s2_sen2cor_nbar/S2B_MSIL2A_20171122T134159_N9999_R124_T22JBM_20211023T090748.SAFE']

#### 3.1.3 Saving Sen2Cor NBAR scene list

In [23]:
s2_sen2cor_nbar_sceneids = toolbox.filename(os.listdir(s2_sen2cor_nbar_dir))
s2_sen2cor_nbar_sceneids[0:3]  # Showing the first three

#### 3.2. Sentinel-2/MSI (LaSRC) NBAR

<!--  -->

The second NBAR correction we will make will be with Sentinel-2/MSI data with LaSRC atmospheric correction. As mentioned earlier in [Section 3.1](#3.1.-Sentinel-2/MSI-(Sen2Cor)-NBAR), the `research processing` library provides three functions for NBAR correction. The first one, presented earlier, corrects Sentinel data generated with Sen2cor. This step will use the second function, named `s2_lasrc_nbar`, which does the NBAR correction on Sentinel data generated with LaSRC.

Let's import this function from the `research processing` library:

In [24]:
from research_processing.nbar import s2_lasrc_nbar

#### 3.2.1 Preparing the Sentinel-2/MSI (LaSRC) NBAR correction output directory

The code below defines based on the Sentinel-2/MSI outputs directory, the path where the generated NBAR data will be stored:

In [25]:
s2_lasrc_nbar_dir = toolbox.prepare_output_directory(s2_outputs, "s2_lasrc_nbar")

#### 3.2.2 Generate NBAR for Sentinel-2 (LaSRC) products

Now we can generate the NBAR on the Surface Reflectance products (generated through LaSRC).

> To generate Sentinel-2 NBAR, the function `s2_lasrc_nbar` uses the [s2-angs](https://github.com/brazil-data-cube/s2-angs) python library. Due to that, here, we do not need to call the angle generation explicitly.

In [ ]:
s2_lasrc_nbar_outputs = s2_lasrc_nbar(lasrc_dir, s2_lasrc_nbar_dir, lasrc_sceneids)
s2_lasrc_nbar_outputs[0:3]  # Showing the first three

#### 3.2.3 Saving LaSRC NBAR scene list

In [ ]:
s2_lasrc_nbar_sceneids = toolbox.filename(os.listdir(s2_sen2cor_nbar_dir))
s2_lasrc_nbar_sceneids[0:3]  # Showing the first three

#### 3.3. Landsat-8/OLI NBAR

Finally, we will do the last NBAR correction in this Jupyter Notebook, which will be done with the Landsat-8/OLI data. We will use the third NBAR correction function provided in the `research processing` library for this step. This function is named `lc8_nbar`, which is imported from the library in the code below:

> In addition to the `lc8_nbar`, the `lc8_generate_angles` function is also imported. Through the [USGS Angle Creation Tool](https://www.usgs.gov/media/files/landsat-8-angles-creation-tools-readme), this function obtains the angles from the Landsat-8/OLI images needed for the NBAR correction.

In [ ]:
from research_processing.nbar import lc8_nbar, lc8_generate_angles

#### 3.3.1 Preparing the Landsat-8 NBAR correction output directory

The code below defines the directory where we will store the generated Landsat-8/OLI NBAR data. The definition is based on the Landsat result directory specified in [Section 1.5](#1.5-Defining-the-base-output-directory).


In [ ]:
lc8_nbar_dir = toolbox.prepare_output_directory(l8_outputs, "lc8_nbar")

#### 3.3.2 Generating Landsat-8 Angles for NBAR calculation

Having defined the output directory, now, using the `lc8_generate_angles` function, we will generate the angles needed for the NBAR correction:

In [ ]:
scene_angles_lc8 = lc8_generate_angles(landsat8_input_dir, landsat8_sceneids)

#### 3.3.3 Generate NBAR for Landsat-8/OLI Surface Reflectance products

Now we can generate the NBAR on the Landsat-8/OLI Surface Reflectance products.

In [ ]:
lc8_nbar_outputs = lc8_nbar(landsat8_input_dir, lc8_nbar_dir, scene_angles_lc8)
lc8_nbar_outputs[0:3]  # Showing the first three

#### 3.3.4 Saving Landsat-8/OLI NBAR scene list

In [ ]:
lc8_nbar_scene_ids = toolbox.filename(os.listdir(lc8_nbar_dir))
lc8_nbar_scene_ids[0:3]  # Showing the first three

## 4. Comparison Analysis
<hr style="border:1px solid #0077b9;">

In this section, we will perform comparative analyses between the data generated in the previous sections. This analysis is divided into two steps. In the first step, we will make comparisons considering each sensor individually, where we will compare scenes from the same sensor in the same region on different dates. Then, in the second step, we will make the comparison between Landsat-8 and Sentinel-2 data together.

For the first step, in the analysis of the individual sensor, we will consider comparisons in the following data:

**Landsat-8**

- NBAR;
- Surface reflectance.

**Sentinel-2 (Sen2cor)**

- NBAR;
- Surface reflectance.

**Sentinel-2 (LaSRC)**

- NBAR;
- Surface reflectance.

As for the analysis between Landsat-8 and Sentinel-2, the following comparisons will be made:

**NBAR**

- Landsat-8 x Sentinel-2 (LaSRC);
- Landsat-8 x Sentinel-2 (Sen2Cor).

**Surface Reflectance**

- Landsat-8 x Sentinel-2 (LaSRC);
- Landsat-8 x Sentinel-2 (Sen2cor).

To do that, first we create pairs of images. This pairing uses images close in time, so for Sentinel-2 we assume that images with a 5 day difference can be compared, while Landsat-8 images within a 10 day difference is also aceptable and when comparing both sensor we also use a maximum difference of 5 days.
The analysis is performed by comparing the pixel values within images overlap and disconsidering pixels masked as cloud, cloud shadow or snow in any of the images being compared. The comparison is performed band-wise.

To perform this step, we will use the functionality contained in the `validation` module of the `research processing` library. From this module, we will import the `validation_funcs` and `validation_routines` sets of functions that have the operations needed to perform the comparison analysis described above.

In [ ]:
from research_processing.validation import validation_funcs
from research_processing.validation import validation_routines

#### 4.1. Surface reflectance comparison: Landsat-8

In this section, the comparison analysis between Landsat-8/OLI Surface Reflectance data is made.

#### 4.1.1 Searching for image pairs

Below we will create pairs of images. Pairs of scenes will be formed that have a sensing date difference of at most 10 days:

In [ ]:
validation_sr_l8_pairs = validation_funcs.search_pairs_l8(landsat8_sceneid_list, day_diff = 10)
validation_sr_l8_pairs[0:3]  # Showing the first three

#### 4.1.2 Preparing the Landsat-8 (SR) analysis output directory

In [ ]:
validation_sr_l8_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_sr_l8")

#### 4.1.3 Defining spectral bands

The code below defines the spectral bands that will be used in the validation.


<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 95%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;"> 
    <b>Important note</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
        <p>In this study, as mentioned, we used Landsat-8 data retrieved through the USGS Earth Explorer. So, the validation function expects the specified band name to be equivalent to the <em>USGS Landsat-8 file band naming pattern</em> (e.g., B1, B2). This naming pattern match is necessary because we will use the name of each of the bands specified to filter the data files. Therefore, if the band name is different from the format defined in the files, the validation function may not find any file to perform the validation.</p>
    </div>
</div>
<br>

In [ ]:
validation_sr_l8_bands = ["B2", "B3", "B4", "B5", "B6", "B7"]

#### 4.1.4 Validate the results

In [ ]:
validation_sr_l8_result = validation_routines.validation_sr_l8(
    input_dir  = landsat8_input_dir,
    cloud_dir  = landsat8_input_dir,
    output_dir = validation_sr_l8_dir,
    pairs      = validation_sr_l8_pairs, 
    bands      = validation_sr_l8_bands
)

### 4.2. Analysis NBAR: Landsat-8

Here the NBAR of Landsat-8 is analysed.

#### 4.2.1 Search pairs

Images with a maximum sensing date difference of 10 days are selected

In [ ]:
validation_nbar_l8_pairs = validation_funcs.search_pairs_l8(landsat8_sceneid_list)

#### 4.2.2 Prepare output directory

In [ ]:
validation_nbar_l8_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_nbar_l8")

#### 4.2.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_bands = ["B2", "B3", "B4", "B5", "B6", "B7"]

#### 4.2.4 Validate the results

In [ ]:
validation_nbar_l8_results = validation_routines.validation_nbar_l8(
    input_dir  = lc8_nbar_dir,
    cloud_dir  = landsat8_input_dir,
    output_dir = validation_nbar_l8_dir,
    pairs      = validation_nbar_l8_pairs,
    bands      = validation_nbar_l8_bands
)

### 4.3. Analysis SR: Sentinel-2 (Sen2Cor)

Here the SR of Sentinel-2 (Sen2cor) is analysed.

#### 4.3.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_sr_s2_sen2cor_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

#### 4.3.2 Prepare output directory

In [ ]:
validation_sr_s2_sen2cor_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_sr_s2_sen2cor")

#### 4.3.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02).

Bands (10m)

In [ ]:
validation_sr_s2_sen2cor_bands10m = ["B02", "B03", "B04", "B08"]

Bands (20m)

In [ ]:
validation_sr_s2_sen2cor_bands20m = ["B8A", "B11", "B12"]

#### 4.3.4 Validate the results

In [ ]:
validation_routines.validation_sr_s2_sen2cor(
    input_dir  = sen2cor_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_sr_s2_sen2cor_dir,
    pairs      = validation_sr_s2_sen2cor_pairs,
    bands10m   = validation_sr_s2_sen2cor_bands10m,
    bands20m   = validation_sr_s2_sen2cor_bands20m
)

### 4.4. Analysis NBAR: Sentinel-2 (Sen2Cor)

Here the NBAR of Sentinel-2 (Sen2cor) is analysed.

#### 4.4.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_nbar_s2_sen2cor_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

#### 4.4.2 Prepare output directory

In [ ]:
validation_nbar_s2_sen2cor_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_nbar_s2_sen2cor")

#### 4.4.3 Defining bands

Name of the spectral bands that will be used in the validation. 

*note:* These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02).

Bands (10m)

In [ ]:
validation_nbar_s2_sen2cor_bands10m = ["B02", "B03", "B04", "B08"]

Bands (20m)

In [ ]:
validation_nbar_s2_sen2cor_bands20m = ["B8A", "B11", "B12"]

#### 4.4.4 Validate the results

In [ ]:
validation_routines.validation_nbar_s2_sen2cor(
    input_dir  = s2_sen2cor_nbar_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_nbar_s2_sen2cor_dir,
    pairs      = validation_nbar_s2_sen2cor_pairs,
    bands10m   = validation_nbar_s2_sen2cor_bands10m,
    bands20m   = validation_nbar_s2_sen2cor_bands20m
)

### 4.5. Analysis SR: Sentinel-2 (LaSRC)

Here the SR of Sentinel-2 (LaSRC) is analysed.

#### 4.5.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_sr_s2_lasrc_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

#### 4.5.2 Prepare output directory

In [ ]:
validation_sr_s2_lasrc_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_sr_s2_lasrc")

#### 4.5.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a)

In [ ]:
validation_sr_s2_lasrc_bands = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

#### 4.5.4 Validate the results

In [ ]:
validation_routines.validation_sr_s2_lasrc(
    input_dir  = lasrc_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_sr_s2_lasrc_dir,
    pairs      = validation_sr_s2_lasrc_pairs,
    bands      = validation_sr_s2_lasrc_bands
)

### 4.6. Analysis NBAR: Sentinel-2 (LaSRC)

Here the NBAR of Sentinel-2 (LaSRC) is analysed.

#### 4.6.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_nbar_s2_lasrc_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

#### 4.6.2 Prepare output directory

In [ ]:
validation_nbar_s2_lasrc_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_nbar_s2_lasrc")

#### 4.6.3 Defining bands

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a)

In [ ]:
validation_nbar_s2_lasrc_bands = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

#### 4.6.4 Validate the results

In [ ]:
validation_routines.validation_nbar_s2_lasrc(
    input_dir  = s2_lasrc_nbar_dir, 
    cloud_dir  = sen2cor_dir, 
    output_dir = validation_nbar_s2_lasrc_dir, 
    pairs      = validation_nbar_s2_lasrc_pairs,
    bands      = validation_nbar_s2_lasrc_bands
)

### 4.7. Analysis SR: Landsat-8 x Sentinel-2 (Sen2Cor)

Here the SR of Landsat-8 and Sentinel-2 (Sen2cor) are analysed.

#### 4.7.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_sr_l8_s2_sen2cor_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

#### 4.7.2 Prepare output directory

In [ ]:
validation_sr_l8_s2_sen2cor_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_sr_l8_s2_sen2cor")

#### 4.7.3 Defining bands

Landsat-8/OLI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_sr_l8_s2_sen2cor_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

Name of the spectral bands (spectral resolution = 10m and 20m) that will be used in the validation.

*note:* These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02, B8A)

In [ ]:
validation_sr_l8_s2_sen2cor_bands_s2 = ["B02", "B03", "B04", "B08", "B8A", "B11", "B12"]

#### 4.7.4 Validate the results

In [ ]:
validation_routines.validation_sr_l8_s2_sen2cor(
    input_dir_l8 = landsat8_input_dir, 
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = sen2cor_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_sr_l8_s2_sen2cor_dir,
    pairs        = validation_sr_l8_s2_sen2cor_pairs,
    bands_l8     = validation_sr_l8_s2_sen2cor_bands_l8,
    bands_s2     = validation_sr_l8_s2_sen2cor_bands_s2
)

### 4.8. Analysis SR: Landsat-8 x Sentinel-2 (LaSRC)

Here the SR of Landsat-8 and Sentinel-2 (LaSRC) are analysed.

#### 4.8.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_sr_l8_s2_lasrc_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

#### 4.8.2 Prepare output directory

In [ ]:
validation_sr_l8_s2_lasrc_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_sr_l8_s2_lasrc")

#### 4.8.3 Defining bands

Landsat-8/OLI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_sr_l8_s2_lasrc_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a).

In [ ]:
validation_sr_l8_s2_lasrc_bands_s2 = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

#### 4.8.4 Validate the results

In [ ]:
validation_routines.validation_sr_l8_s2_lasrc(
    input_dir_l8 = landsat8_input_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = lasrc_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_sr_l8_s2_lasrc_dir,
    pairs        = validation_sr_l8_s2_lasrc_pairs,
    bands_l8     = validation_sr_l8_s2_lasrc_bands_l8,
    bands_s2     = validation_sr_l8_s2_lasrc_bands_s2
)

### 4.9. Analysis NBAR: Landsat-8 x Sentinel-2 (Sen2Cor)

Here the NBAR of Landsat-8 and Sentinel-2 (Sen2cor) are analysed.

#### 4.9.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_nbar_l8_s2_sen2cor_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

#### 4.9.2 Prepare output directory

In [ ]:
validation_nbar_l8_s2_sen2cor_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_nbar_l8_s2_sen2cor")

#### 4.9.3 Defining bands

Landsat-8/OLI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_s2_sen2cor_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

Name of the spectral bands (spectral resolution = 10m and 20m) that will be used in the validation.

*note:* These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02, B8A)

In [ ]:
validation_nbar_l8_s2_sen2cor_bands_s2 = ["B02", "B03", "B04", "B08", "B8A", "B11", "B12"]

#### 4.9.4 Validate the results

In [ ]:
validation_routines.validation_nbar_l8_s2_sen2cor(
    input_dir_l8 = lc8_nbar_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = s2_sen2cor_nbar_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_nbar_l8_s2_sen2cor_dir,
    pairs        = validation_nbar_l8_s2_sen2cor_pairs,
    bands_l8     = validation_nbar_l8_s2_sen2cor_bands_l8,
    bands_s2     = validation_nbar_l8_s2_sen2cor_bands_s2
)

### 4.10. Analysis NBAR Landsat-8 x Sentinel-2 (LaSRC)

Here the NBAR of Landsat-8 and Sentinel-2 (LaSRC) are analysed.

#### 4.10.1 Search pairs

Images with a maximum sensing date difference of 5 days are selected

In [ ]:
validation_nbar_l8_s2_lasrc_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

#### 4.10.2 Prepare output directory

In [ ]:
validation_nbar_l8_s2_lasrc_dir = toolbox.prepare_output_directory(analysis_outputs, "validation_nbar_l8_s2_lasrc")

#### 4.10.3 Defining bands

Landsat-8/OLI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_s2_lasrc_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

Name of the spectral bands that will be used in the validation.

*note:* These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a).

In [ ]:
validation_nbar_l8_s2_lasrc_bands_s2 = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

#### 4.10.4 Validate the results

In [ ]:
validation_routines.validation_nbar_l8_s2_lasrc(
    input_dir_l8 = lc8_nbar_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = s2_lasrc_nbar_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_nbar_l8_s2_lasrc_dir, 
    pairs        = validation_nbar_l8_s2_lasrc_pairs,
    bands_l8     = validation_nbar_l8_s2_lasrc_bands_l8,
    bands_s2     = validation_nbar_l8_s2_lasrc_bands_s2
)

## 5. Saving results table
<hr style="border:1px solid #0077b9;">

In this section, the analysis results are inserted into a tidy structure and saved as a `.csv`. 

In [ ]:
from research_processing.tidy import map_validation_folder_as_tidydata

#### 5.1 Generating the tidy results

In [ ]:
tidy_output = map_validation_folder_as_tidydata(analysis_outputs)
tidy_output

5.2 Saving the results

In [ ]:
tidy_output.to_csv(os.path.join(analysis_outputs, "results.csv"))